In [58]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import os
import numpy
import pandas as pd
import math
import urllib
import xlwt
import keyboard

In [59]:
os.chdir('D:/AI')

In [60]:
os.getcwd()

'D:\\AI'

저장 장소 변경 후 확인

In [61]:
os.chdir('D:/AI/DaTa')
os.getcwd()

'D:\\AI\\DaTa'

현재시간으로 폴더를 만들기 위해 현재시간 저장(이렇게하면 폴더명이 같아 에러가 발생하는 경우를 없애줄 수 있음.)

In [62]:
now=time.localtime()

현재시간으로 폴더 생성 후 생성한 폴더로 저장 경로 변경

In [63]:
t1='Coupang'
t2='%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
dir_name = t1 + t2
print(dir_name)

os.mkdir(dir_name)
os.chdir(dir_name)
os.getcwd()

Coupang2020-11-02-15-55-13


'D:\\AI\\DaTa\\Coupang2020-11-02-15-55-13'

상품의 정보를 저장하기 위한 폴더 생성

In [64]:
name1='Info_of_products'
name2='pics_of_products'
os.mkdir(name1)
os.getcwd()

'D:\\AI\\DaTa\\Coupang2020-11-02-15-55-13'

In [65]:
adr=os.getcwd()

제품사진만을 따로 볼수 있게 폴더를 따로 만들고 사용자가 사진 크롤링 여부를 직접 선택할 수 있게 만듦.

In [66]:
query_txt = input('1.찾아보고 싶은 상품은 무엇입니까?: ')
cnt = int(input('2.크롤링 할 데이터 개수를 쓰시오: '))
page_cnt = math.ceil(cnt/36)
query_txt2 = input('3.우선 순위를 선택하세요.(쿠팡랭킹순/낮은가격순/높은가격순/판매량순/최신순)')
pic = input('4.사진도 함께 크롤링하기를 원하십니까?(y/n)')
while (pic != 'y') and (pic != 'n'):
    pic = input('-----y 나 n 만 입력가능합니다. 다시 입력하세요----- ')
if pic == 'y':
    os.mkdir(name2)
    os.chdir(name1)
fx_name = input('5.xlsx 형태로 저장할 파일명을 확장자 포함해서 쓰세요.(예:Info_of_products.xlsx): ')

1.찾아보고 싶은 상품은 무엇입니까?: 머그컵
2.크롤링 할 데이터 개수를 쓰시오: 100
3.우선 순위를 선택하세요.(쿠팡랭킹순/낮은가격순/높은가격순/판매량순/최신순)판매량순
4.사진도 함께 크롤링하기를 원하십니까?(y/n)y
5.xlsx 형태로 저장할 파일명을 확장자 포함해서 쓰세요.(예:Info_of_products.xlsx): Info_of_products.xlsx


크롬드라이버를 이용해 웹 브라우저 열기

In [67]:
path = "c:/Temp/chromedriver_240/chromedriver.exe"
driver=webdriver.Chrome(path)

driver.get('http://www.coupang.com')
s_time=time.time()

찾아보고 싶은 상품 검색

In [68]:
element=driver.find_element_by_id('headerSearchKeyword')
element.send_keys(query_txt)
element.submit()

사용자의 우선순위에 따라 검색 결과 우선순위 설정

In [69]:
if query_txt2 == '낮은가격순':
    driver.find_element_by_xpath('//*[@id="searchSortingOrder"]/ul/li[2]/label').click()
elif query_txt2 == '높은가격순':
    driver.find_element_by_xpath('//*[@id="searchSortingOrder"]/ul/li[3]/label').click()
elif query_txt2 == '판매량순':
    driver.find_element_by_xpath('//*[@id="searchSortingOrder"]/ul/li[4]/label').click()
elif query_txt2 =='최신순':
    driver.find_element_by_xpath('//*[@id="searchSortingOrder"]/ul/li[5]/label').click()


입력한 cnt 개수만큼 크롤링하기(위의 사진 크롤링에 관한 질문에 y라 대답했으면 사진도 같이 크롤링하여 사진 폴더에 담기)

In [70]:
no2 = []
p_name2 = []
p_price2 = []
p_d_date2 = []
p_img2 = []
no=1
num=1

for x in range(1,page_cnt+1):
    time.sleep(2)
    print("="*80)
    print("● %s 페이지의 상품정보" %x)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    p_list = soup.select('ul > li > a > dl')
    imgs=driver.find_elements_by_class_name('search-product-wrap-img')
    
    for i in p_list:
        no2.append(no)
        print("1.번호 : %s" %no)
        
        
        p_name = i.find('div', 'name').get_text()
        p_name2.append(p_name)
        print("2.상품명 : " +p_name)
        #품절되어 가격정보, 배송정보가 없는 경우 품절되었다는 텍스트를 통해 알려줌.
        try:
            p_price = i.find('strong', 'price-value').get_text()
            p_price2.append(p_price)
            print("3.가격 : "+ p_price + '원')
        except AttributeError:
            print('해당상품은 품절되었습니다.')
            p_price2.append('해당상품은 품절되었습니다. ')
        
        try:
            p_d_date = i.find('div', 'delivery').get_text()
            p_d_date2.append(p_d_date)
            print("4.배송예정일 : " + p_d_date )
        except AttributeError:
            print('해당상품은 품절되었습니다.')
            p_d_date2.append('해당상품은 품절되었습니다.')
            
            
        print('\n')
        
        
        if no == cnt:
            break
        no+=1
    os.chdir(adr)
#만약 이미지 정보를 수집한다고 했으면 이미지 정보를 크롤링해 리스트로 만들기
    if pic == 'y':
        for i in imgs:
            p_img = i.get_attribute('src')
            p_img2.append(p_img)
        
            if num == cnt:
                break
            num+=1
    time.sleep(2)
#페이지 넘기기, 만약 더이상 넘어갈 페이지가 없다면 없다는 표시 해주기
    x+=1
    if x == page_cnt+1:
        break
    try :    
        driver.find_element_by_link_text('''%s'''%x).click()
    except :
        print('\n 더 이상 해당 키워드를 포함하고 있는 상품이 존재하지 않습니다. ')
        break   


● 1 페이지의 상품정보
1.번호 : 1
2.상품명 : 릿첼 AQ 첫걸음 머그 스트로, 옐로우, 1개
3.가격 : 11,680원
4.배송예정일 :  내일(화) 11/3   도착 보장  


1.번호 : 2
2.상품명 : 글라스락 손잡이 빅 머그컵 510ml, 2개입
3.가격 : 5,210원
4.배송예정일 :  내일(화) 11/3   도착 보장  


1.번호 : 3
2.상품명 : 릿첼 AQ 첫걸음 머그 스트로, 핑크, 1개
3.가격 : 11,680원
4.배송예정일 :  내일(화) 11/3   도착 보장  


1.번호 : 4
2.상품명 : 락앤락 메트로 보온보냉 머그 LHC4219, MINT, 475ml
3.가격 : 14,260원
4.배송예정일 :  내일(화) 11/3   도착 보장  


1.번호 : 5
2.상품명 : 아이조움 핑크퐁 입체 머그컵, 혼합색상, 1개
3.가격 : 3,000원
4.배송예정일 :  내일(화) 11/3   도착 보장  


1.번호 : 6
2.상품명 : 오슬로 아웃백 이중진공 스텐머그컵 340ml, 메탈, 1개
3.가격 : 7,870원
4.배송예정일 :  내일(화) 11/3   도착 보장  


1.번호 : 7
2.상품명 : JVR 유로 머그컵 4종 세트 280ml, 화이트, 핑크, 블루, 퍼플, 1세트
3.가격 : 20,370원
4.배송예정일 :  내일(화) 11/3   도착 보장  


1.번호 : 8
2.상품명 : 벨루즈까사 인디언머그 5종 세트, 오렌지, 베이지, 스카이, 레드, 블루, 1세트
3.가격 : 13,530원
4.배송예정일 :  내일(화) 11/3   도착 보장  


1.번호 : 9
2.상품명 : 릿첼 AQ 슬림머그 교체용 빨대 세트 S-5, 단일 색상, 1세트
3.가격 : 5,700원
4.배송예정일 :  내일(화) 11/3   도착 보장  


1.번호 : 10
2.상품명 : 보노스프 5종 6각 머그컵 (총 18봉입), 1세트
3.가격 : 18,900원
4.배송예정일 :  목요일 11/5   도착 예정  


1

In [71]:
if pic == 'y':
    os.chdir(name2)
os.getcwd()

'D:\\AI\\DaTa\\Coupang2020-11-02-15-55-13\\pics_of_products'

In [72]:

if pic == 'y':
    img_no = 1
    
    for i in range(0,len(p_img2)):
        try :
            urllib.request.urlretrieve(p_img2[i],'product'+ str(img_no)+'.jpg')     
#img 부분이 존재하지 않을 경우를 대비
        except TypeError:    
            continue   
        time.sleep(1)
        img_no+=1

In [73]:
Coupang_search = pd.DataFrame()
Coupang_search['번호']=no2
Coupang_search['상품명']=p_name2
Coupang_search['가격']=p_price2
Coupang_search['배송예정일']=p_d_date2
if pic == 'y':
    Coupang_search['사진링크']=p_img2

In [74]:
Coupang_search

,번호,상품명,가격,배송예정일,사진링크
0,1,"릿첼 AQ 첫걸음 머그 스트로, 옐로우, 1개","11,680",내일(화) 11/3 도착 보장,https://thumbnail6.coupangcdn.com/thumbnails/r...
1,2,"글라스락 손잡이 빅 머그컵 510ml, 2개입","5,210",내일(화) 11/3 도착 보장,https://thumbnail8.coupangcdn.com/thumbnails/r...
2,3,"릿첼 AQ 첫걸음 머그 스트로, 핑크, 1개","11,680",내일(화) 11/3 도착 보장,https://thumbnail7.coupangcdn.com/thumbnails/r...
3,4,"락앤락 메트로 보온보냉 머그 LHC4219, MINT, 475ml","14,260",내일(화) 11/3 도착 보장,https://thumbnail8.coupangcdn.com/thumbnails/r...
4,5,"아이조움 핑크퐁 입체 머그컵, 혼합색상, 1개","3,000",내일(화) 11/3 도착 보장,https://thumbnail6.coupangcdn.com/thumbnails/r...
...,...,...,...,...,...
95,96,"[바보사랑]오션글라스 뮌헨 비어머그 355ml 6P, 6개입","17,200",내일(화) 11/3 도착 보장,https://thumbnail9.coupangcdn.com/thumbnails/r...
96,97,"카카오프렌즈 페이스 캡롱 머그 410ml, 어피치, 1개","9,870",내일(화) 11/3 도착 보장,https://thumbnail9.coupangcdn.com/thumbnails/r...
97,98,"머그앤키친 칼라 시리즈 머그컵 60종, MK일자화이트, 1개","1,000",모레(수) 11/4 도착 예정,https://thumbnail7.coupangcdn.com/thumbnails/r...
98,99,"오엑스코 마이 프린세스 키즈 식기 3p 세트, 혼합 색상, 공기 + 대접 + 머그컵","9,900",내일(화) 11/3 도착 보장,https://thumbnail9.coupangcdn.com/thumbnails/r...


In [75]:
os.chdir(adr)
os.chdir(name1)
os.getcwd()

Coupang_search.to_excel(fx_name)

In [76]:
e_time=time.time()
t_time=e_time - s_time

In [77]:
ads_imgs = os.getcwd()
print('='*88)
print("크롤링을 요청한 총 %s 건 중에서 %s 건의 데이터를 수집완료 했습니다. \n총 소요시간은 %s 초입니다. " %(cnt, no, round(t_time,1)))
print('파일 저장 완료: xlsx 파일명 : ', fx_name)
if pic == 'y' :
    print('제품 사진을 전체적으로 확인하고 싶으면 ', ads_imgs, '에서 확인하세요' )
print("="*88)

크롤링을 요청한 총 100 건 중에서 100 건의 데이터를 수집완료 했습니다. 
총 소요시간은 333.2 초입니다. 
파일 저장 완료: xlsx 파일명 :  Info_of_products.xlsx
제품 사진을 전체적으로 확인하고 싶으면  D:\AI\DaTa\Coupang2020-11-02-15-55-13\Info_of_products 에서 확인하세요


In [78]:
driver.close()

😀느낀점

평소에 자주 사용하던 쿠팡에서 간단한 제품정보를 한눈에 볼 수 있는 프로그램이 있으면 편할 것이라 생각해서 이 주제로 코딩을 하게 외었습니다. 코딩을 하며 웹페이지의 구성에 대해 좀 더 자세히 알 수 있었고 수업시간에 배웠던 내용을 이 과제를 통해 총정리 하는 느낌이 들었습니다. 또 어떻게하면 사소한 부분에서 사용자가 더 편의를 느낄까? 어떤 기능을 추가하면 좋을까? 라는 고민을 하며 코딩을 좀 더 진지하게 바로 볼 수 있게 된 것 같습니다. 제 꿈이 아직 정해지지는 않았지만, 컴퓨터 쪽으로 가고 싶다는 생각을 하고 있었는데, 이번 과제를 통해 코딩하는 즐거움도 느끼고 크롤링에 대해 정리를 하는 느낌이라 좋았습니다. 